In [ ]:
%matplotlib notebook
import sys
sys.path.insert(0, '..')

import math
import matplotlib as mp
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal

import dragonradio.liquid
import dragonradio.packet
from dragonradio.tools.modem import *
from dragonradio.tools.plot.radio import PSDPlot

In [ ]:
# Sampling frequency
Fs = 10e6

# Channel bandwidth
cbw = 1e6

In [ ]:
# Modulate first packet
hdr1 = dragonradio.packet.Header(1, 2, 0)
mcs1 = dragonradio.liquid.MCS('crc32', 'none', 'v27', 'bpsk')

# Center frequency shift
fc1 = 1e6

payload1 = b'0' * 1500

sig1 = modulateMix(hdr1, mcs1, payload1, cbw, fc1, Fs)

# Plot PSD of combined signal
fig = PSDPlot(*plt.subplots(), nfft=1024)
fig.plot(Fs, sig1)

In [ ]:
# Demodulate first packet
demodulateMix(sig1, cbw, fc1, Fs)

In [ ]:
# Modulate second packet
hdr2 = dragonradio.packet.Header(2, 1, 0)
mcs2 = dragonradio.liquid.MCS('crc32', 'rs8', 'none', 'qam256')

# Center frequency shift
fc2 = 2e6

payload2 = b'0' * 1500

sig2 = modulateMix(hdr2, mcs2, payload2, cbw, fc2, Fs)

# Plot PSD of combined signal
fig = PSDPlot(*plt.subplots(), nfft=1024)
fig.plot(Fs, sig2)

In [ ]:
# Demodulate second packet
demodulateMix(sig2, cbw, fc2, Fs)